<a href="https://colab.research.google.com/github/ArsT1982/Spark_DZ2/blob/main/Spark_SQL_DZ_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ДЗ-2 (Spark SQL)


---



Подготовка к работе с файлом "covid-data.csv" и создание Spark сессии.

In [1]:
pip install --quiet pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()

In [3]:
from pyspark import SparkFiles

covid_data_file_url = "https://raw.githubusercontent.com/glincow/netology-spark-sql/main/data/covid-data.csv"
spark.sparkContext.addFile(covid_data_file_url)
file_path  = 'file://' + SparkFiles.get('covid-data.csv')
df = spark.read.option('inferSchema', 'true').option('header', 'true').csv(file_path)

Подготовка датасета к работе:
Анализ данных и значений в исходном файле.

Обнаружил:
1. В файле присутствуют null - заменяю эти значения на (0) в датасете.
2. В файле присутствуют данные по Евросоюзу (и др. варианты данных вида "OWID_%" в поле 'iso_code').
Для аналитики важны данные по странам - их и оставил, отсальное исключил из датасета.


In [4]:
from pyspark.sql import functions as F

Find_OWID_from_df = df.filter(F.col('iso_code').like('OWID_%'))
Exclude_OWID_from_df = df.subtract(Find_OWID_from_df)

df_null_to_zero=Exclude_OWID_from_df.na.fill(0)

Задание-1.


---

Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших).


---
Важно! Посчитал, что переболевшие (alias 'процент переболевших') - это те, кто входит в кол-во в графе общего количества заболевших ('total_cases'), но не относится к графе умерших ('total_deaths'). ("Переболел - значит выжил")

Посчитал как частное от деления на популяцию, умноженное на 100 (для процентов) и округлил до второго знака.
Данные по Евросоюзу были исколючены на предыдущих шагах.

In [5]:
from pyspark.sql import functions as F

df_perc_recov_by_cntry = df_null_to_zero.select('iso_code', F.col('location').alias('страна'),F.round(((F.col('total_cases')-F.col('total_deaths'))*100.0/F.col('population')),2).alias('процент переболевших')).filter(df_null_to_zero.date == '2020-03-31').sort(F.col('процент переболевших').desc())
df_perc_recov_by_cntry.show(15)

+--------+-------------+--------------------+
|iso_code|       страна|процент переболевших|
+--------+-------------+--------------------+
|     VAT|      Vatican|                0.74|
|     SMR|   San Marino|                0.62|
|     AND|      Andorra|                0.47|
|     LUX|   Luxembourg|                0.34|
|     ISL|      Iceland|                0.33|
|     CHE|  Switzerland|                0.19|
|     ESP|        Spain|                0.19|
|     LIE|Liechtenstein|                0.18|
|     ITA|        Italy|                0.15|
|     MCO|       Monaco|                0.13|
|     AUT|      Austria|                0.11|
|     BEL|      Belgium|                 0.1|
|     DEU|      Germany|                0.08|
|     NOR|       Norway|                0.08|
|     FRA|       France|                0.07|
+--------+-------------+--------------------+
only showing top 15 rows



Задание-2.


---

Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)


---


Отфильтровал весь файл по нужным нам датам 4-ой недели марта 2021.
Нашел максимумы кол-ва новых случаев (F.max) применяя оконную функцию партициями по странам.
Далее сделал выборку из датасета сравнением с этими ненулевыми максимумами. После этого вывел TOP 10 стран.

In [48]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w = Window.partitionBy("страна")
df_new_cases_week = df_null_to_zero.select('iso_code', F.col('location').alias('страна'), F.col('date').alias('дата'), F.col('new_cases').alias('кол-во новых случаев')).filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).sort(F.col('страна').asc())
Df_window = df_new_cases_week.withColumn("new_col", F.max('кол-во новых случаев').over(w)).sort(F.col('страна').asc())
df_result = Df_window.select('страна','дата','кол-во новых случаев').where((F.col('кол-во новых случаев') == F.col('new_col')) & (F.col('new_col') != 0)).sort(F.col('кол-во новых случаев').desc())
df_result.show(10)


+-------------+----------+--------------------+
|       страна|      дата|кол-во новых случаев|
+-------------+----------+--------------------+
|       Brazil|2021-03-25|            100158.0|
|United States|2021-03-24|             86960.0|
|        India|2021-03-28|             68020.0|
|       France|2021-03-24|             65392.0|
|       Poland|2021-03-26|             35145.0|
|       Turkey|2021-03-27|             30021.0|
|        Italy|2021-03-22|             24501.0|
|      Germany|2021-03-24|             23757.0|
|         Peru|2021-03-25|             19206.0|
|      Ukraine|2021-03-26|             18226.0|
+-------------+----------+--------------------+
only showing top 10 rows





---



Задание-3.

Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в России вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

Использовал оконную функцию lag для нахождения значения за предыдущий день (alias 'кол-во новых случаев вчера').
Для корректного расчета первого дня в выборке в исходном селекте использовал более шировкий диапазон дат - с 21 марта для исключения нуля в результате ф-ции lag.
В итоговом результате оставлен период за 4ю неделю с 22 по 28 марта 2021г.
Разница с предыдущим днем - alias 'дельта'

In [7]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

w = Window.orderBy("date")
select_for_df = df_null_to_zero.select('iso_code', 'location', 'date', F.col('new_cases').alias('кол-во новых случаев сегодня')).filter(df_null_to_zero.date.between ('2021-03-21','2021-03-28')).filter(F.col('location').like('Russia')).sort(F.col('date').asc())

df_itog = select_for_df.withColumn("кол-во новых случаев вчера", lag('кол-во новых случаев сегодня', 1, 0).over(w))
df_result = df_itog.select('date','кол-во новых случаев сегодня','кол-во новых случаев вчера',(F.col('кол-во новых случаев сегодня')-F.col('кол-во новых случаев вчера')).alias('дельта')).filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28'))
df_result2 = df_result.select(F.col('date').alias('дата'),'кол-во новых случаев сегодня','кол-во новых случаев вчера','дельта').show()


+----------+----------------------------+--------------------------+------+
|      дата|кол-во новых случаев сегодня|кол-во новых случаев вчера|дельта|
+----------+----------------------------+--------------------------+------+
|2021-03-22|                      9195.0|                    9215.0| -20.0|
|2021-03-23|                      8369.0|                    9195.0|-826.0|
|2021-03-24|                      8769.0|                    8369.0| 400.0|
|2021-03-25|                      9128.0|                    8769.0| 359.0|
|2021-03-26|                      9073.0|                    9128.0| -55.0|
|2021-03-27|                      8783.0|                    9073.0|-290.0|
|2021-03-28|                      8979.0|                    8783.0| 196.0|
+----------+----------------------------+--------------------------+------+

